In [1]:
import numpy as np
import cv2
import time
import os
from matplotlib import pyplot as plt
import mediapipe as mp

In [2]:
# Open the webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():

    # Read the feed
    ret, frame = cap.read()

    if not ret:
        print("Failed to grab frame")
        break

    # Show the frame on the screen
    cv2.imshow("OpenCV Feed", frame)

    # Check if 'q' key is pressed to exit
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()